# PHS564 — Lecture 10 (Student)
## Causal survival analysis: time-to-event outcomes, censoring, discrete-time hazards (MIMIC-IV Demo)

### Learning goals
- Distinguish **risk** vs **hazard** vs **survival**; know what effect measure you are estimating.
- Convert a cohort into **person-period** (discrete time) data and fit pooled logistic hazards.
- Handle **right censoring** with **IPCW**; understand when censoring is “informative”.
- Produce causal survival curves under a point treatment (and interpret assumptions).

### Required reading
- Hernán & Robins, sections on censoring and survival (target trial chapters/sections as applicable).


In [ ]:
# Colab bootstrap (run this first if you opened from a Colab badge)
# - Clones the repo into /content/PHS564 (if needed)
# - Installs requirements
# - Adds repo to sys.path

from __future__ import annotations

import os
import sys
import subprocess
from pathlib import Path


def _in_colab() -> bool:
    return "google.colab" in sys.modules


if _in_colab():
    REPO_URL = "https://github.com/vafaei-ar/PHS564.git"
    TARGET_DIR = Path("/content/PHS564")

    if not (TARGET_DIR / "requirements.txt").exists():
        print("Cloning course repo into Colab runtime...")
        subprocess.run(["git", "clone", "--depth", "1", REPO_URL, str(TARGET_DIR)], check=True)

    os.chdir(TARGET_DIR)

    print("Installing requirements...")
    subprocess.run([sys.executable, "-m", "pip", "-q", "install", "-r", "requirements.txt"], check=True)

    if str(TARGET_DIR) not in sys.path:
        sys.path.insert(0, str(TARGET_DIR))

    print("✓ Colab setup complete. Now run the rest of the notebook.")
else:
    print("Not running in Colab; skipping Colab bootstrap.")


### Setup

This notebook is designed to run **locally** or in **Google Colab**.

**Colab workflow (recommended):**
1) Clone the course repo (ask the instructor for the GitHub URL).
2) Install requirements.
3) Run the notebook top-to-bottom.

> If you opened this notebook directly from GitHub in Colab (without cloning),
> relative paths will not work. Clone first.


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility
RNG = np.random.default_rng(564)

# Locate repo root (works when running from lectures/Lxx.../student or /instructor)
THIS_DIR = Path.cwd()
REPO_ROOT = THIS_DIR
for _ in range(4):
    if (REPO_ROOT / "requirements.txt").exists() or (REPO_ROOT / "README.md").exists():
        break
    REPO_ROOT = REPO_ROOT.parent

DATA_DIR = REPO_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"

print("Working directory:", THIS_DIR)
print("Repo root:", REPO_ROOT)
print("Processed data dir exists:", PROC_DIR.exists())


### Download the processed cohort extract (required)

This lecture uses a **processed cohort extract** hosted as a **GitHub Release asset**.

- If you are running in **Google Colab**, run the next cell once to download the processed cohort into `data/processed/`.
- Downloading raw MIMIC-IV Demo tables is optional and **not required** for this lecture.

In [ ]:
# Download processed cohort extract for L10 (safe to re-run)
# Downloads into data/processed/ from GitHub Releases.

try:
    from data.download_data import download_course_extracts

    download_course_extracts("L10", out_dir=PROC_DIR)
except Exception as e:
    print("Could not download the processed cohort extract for L10.")
    print("Error:", e)
    print("If you already have the cohort file, place it in data/processed/ and re-run.")

### Optional: download raw MIMIC-IV Demo tables

Not required for the homework pipeline. Skip unless your instructor asks you to explore the raw Demo tables in `data/raw/`.

In [ ]:
# Optional: download raw MIMIC-IV Demo tables to data/raw/
# Not required for the processed-cohort pipeline.

try:
    from data.download_data import download_mimic_demo

    download_mimic_demo(out_dir=RAW_DIR, version="2.2", method="python")
except Exception as e:
    print("Skipping raw MIMIC-IV Demo download.")
    print("Error:", e)

## Data
This lecture uses a **processed cohort extract** derived from MIMIC-IV Demo.

Expected file: `data/processed/cohort_L10_survival.parquet` (or `.csv`).

If this file is missing:
- In Colab: run the “Download the processed cohort extract” cell above.
- Or locally: run `python data/download_data.py --lecture L10` (downloads into `data/processed/`).

Assumed columns (you may adapt to the actual extract):
- `A` treatment at baseline
- `T` follow-up time (in discrete intervals)
- `E` event indicator (1=event, 0=censored)
- baseline covariates (e.g., age, sex, severity)

We will fit a **discrete-time hazard** model via logistic regression on a person-period dataset.


In [ ]:
# Statsmodels for regression (logit/ols); installed via requirements.txt
import statsmodels.api as sm
import statsmodels.formula.api as smf

parquet_path = PROC_DIR / "cohort_L10_survival.parquet"
csv_path = PROC_DIR / "cohort_L10_survival.csv"

if parquet_path.exists():
    df = pd.read_parquet(parquet_path)
elif csv_path.exists():
    df = pd.read_csv(csv_path)
else:
    raise FileNotFoundError(
        "Missing cohort file for L10. Download it via:\n"
        "  - Colab: run the download cell above\n"
        "  - Local: python data/download_data.py --lecture L10\n"
    )

df.head()

### TODO A1 — Create person-period dataset
If your extract is already long-format (one row per person-period), skip this.
Otherwise, expand each subject into rows `t=1..T` and set event indicator at the event time.


In [ ]:
# Expected: either already long with columns ['id','t','A','E_t', ...]
# or wide with ['id','T','E', ...]
# TODO: adapt based on df columns.

long = df.copy()  # placeholder
long.head()

## Part B — Discrete-time hazard model
We model Pr(E_t=1 | E_{t-1}=0, A, L, t).


In [ ]:
# TODO: set outcome and covariates
# Recommended structure:
#   hazard_formula = "E_t ~ A + age + sex + C(t)"  (with t as categorical or spline)
hazard_formula = "E_t ~ A + t"  # TODO
# Fit model
haz_model = smf.logit(hazard_formula, data=long).fit(disp=False)
haz_model.params.head()

### TODO B1 — Predict survival curves under A=1 and A=0 (g-formula)
Compute survival S(t) = Π_{k<=t} (1 - h(k)).


In [ ]:
def survival_curve(data_long: pd.DataFrame, A_value: int, t_col: str = "t") -> pd.DataFrame:
    # data_long contains one row per person-period and baseline covariates.
    d = data_long.copy()
    d["A"] = A_value
    d["haz"] = haz_model.predict(d)
    # average hazard per time
    hz = d.groupby(t_col)["haz"].mean().sort_index()
    surv = (1 - hz).cumprod()
    return pd.DataFrame({"t": hz.index, "haz": hz.values, "surv": surv.values})

# TODO: adapt t column name if needed
curve1 = survival_curve(long, 1)
curve0 = survival_curve(long, 0)
curve1.head(), curve0.head()

In [ ]:
plt.figure()
plt.step(curve0["t"], curve0["surv"], where="post", label="A=0")
plt.step(curve1["t"], curve1["surv"], where="post", label="A=1")
plt.xlabel("time interval")
plt.ylabel("Survival S(t)")
plt.legend()
plt.title("Estimated survival curves (discrete-time)")
plt.show()

## Reflection
1) Why is time-to-event analysis central to target trial emulation?
2) What is the difference between hazard and risk, and why does it matter?
